In [1]:
import pandas as pd
import numpy as np

In [2]:
practice_data = pd.read_csv("Downloads/airpollution/data/GPsize.csv")
practice_address_data = pd.read_csv("Downloads/airpollution/data/T201912ADDR BNFT.csv", names = ["date", "Code", "Name", "Name2", "Add", "Loc", "Place", "Postcode"])
pollution_data = pd.read_csv("Downloads/airpollution/data/mappm252019g.csv")
prescription_data = pd.read_csv("Downloads/airpollution/data/T201912PDPI BNFT.csv")

In [3]:
prescription_data.rename(columns = {"PRACTICE":"Code"}, inplace = True)
prescription_data = prescription_data.merge(practice_data.drop(columns=["Practice"]))

In [4]:
import pgeocode
from convertbng.util import convert_bng
import tqdm

practice_address_data = practice_address_data.drop(columns=["date", "Name", "Name2", "Add", "Loc", "Place"])

nomi = pgeocode.Nominatim('gb')
practice_address_data["x"], practice_address_data["y"] = 0, 0


for i in tqdm.tqdm(range(len(practice_address_data))):
    postcode = practice_address_data["Postcode"][i]
    if pd.isnull(postcode): continue
    loc = nomi.query_postal_code(practice_address_data["Postcode"][i])
    x,y = convert_bng(loc["longitude"], loc["latitude"])
    if pd.isnull(x) or pd.isnull(y): continue
    practice_address_data["x"][i] = int(round(x[0], -3)) + 500
    practice_address_data["y"][i] = int(round(y[0], -3)) + 500
    
prescription_data = prescription_data.merge(practice_address_data)

  0%|          | 0/9324 [00:00<?, ?it/s]/Users/hattie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/hattie/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 9324/9324 [00:44<00:00, 207.32it/s]


In [5]:
pd.isnull(practice_address_data["Postcode"][4807])

True

In [6]:
prescription_data["Total List Size"] = prescription_data["Total List Size"].str.replace(',', '').astype(float)
prescription_data

,SHA,PCT,Code,BNF CODE,BNF NAME,ITEMS,NIC,ACT COST,QUANTITY,PERIOD,Total List Size,Postcode,x,y
0,Q44,01C,N81002,0101010G0BCABAB,Mucogel_Susp 195mg/220mg/5ml S/F,1,2.99,2.79,500.0,201912,13040.0,SK9 1PA,385500,379500
1,Q44,01C,N81002,0101021B0AAALAL,Sod Algin/Pot Bicarb_Susp S/F,5,26.11,24.53,2550.0,201912,13040.0,SK9 1PA,385500,379500
2,Q44,01C,N81002,0101021B0AAAPAP,Sod Alginate/Pot Bicarb_Tab Chble 500mg,1,3.07,2.87,60.0,201912,13040.0,SK9 1PA,385500,379500
3,Q44,01C,N81002,0101021B0BEACAH,Gaviscon_Liq Orig Aniseed Relief,1,7.11,6.62,600.0,201912,13040.0,SK9 1PA,385500,379500
4,Q44,01C,N81002,0101021B0BEADAJ,Gaviscon Infant_Oral Pdr Sach,3,33.74,31.40,210.0,201912,13040.0,SK9 1PA,385500,379500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9154947,Q70,99M,J82630,23804708006,Smith & Nephew_Proshield Foam & Spy Skin Clean...,4,26.44,24.61,4.0,201912,5302.0,GU14 6DH,484500,156500
9154948,Q70,99M,J82630,23804708007,Smith & Nephew_Proshield Plus Skin Prote 115g,3,29.82,27.74,3.0,201912,5302.0,GU14 6DH,484500,156500
9154949,Q70,99M,J82630,23854808504,ConvaTec_Stomahesive Seal 48mm x 3mm,1,38.42,35.71,20.0,201912,5302.0,GU14 6DH,484500,156500
9154950,Q70,99M,J82630,23944809487,ConvaTec_Combihesive Natura Flx Flng 45mm,1,33.82,31.45,10.0,201912,5302.0,GU14 6DH,484500,156500


In [7]:
prescription_data["RATE"] = prescription_data["QUANTITY"] / prescription_data["Total List Size"]

In [8]:
prescription_data = prescription_data.merge(pollution_data)
prescription_data = prescription_data.replace('MISSING',np.NaN)
prescription_data["pm252019g"] = pd.to_numeric(prescription_data["pm252019g"], downcast="float")

In [9]:
prescription_data["BNF CHAPTER"] = prescription_data["BNF CODE"].astype(str).str[0:2]

In [10]:
prescription_data

,SHA,PCT,Code,BNF CODE,BNF NAME,ITEMS,NIC,ACT COST,QUANTITY,PERIOD,Total List Size,Postcode,x,y,RATE,ukgridcode,pm252019g,BNF CHAPTER
0,Q44,01C,N81002,0101010G0BCABAB,Mucogel_Susp 195mg/220mg/5ml S/F,1,2.99,2.79,500.0,201912,13040.0,SK9 1PA,385500,379500,0.038344,635196,7.980608,01
1,Q44,01C,N81002,0101021B0AAALAL,Sod Algin/Pot Bicarb_Susp S/F,5,26.11,24.53,2550.0,201912,13040.0,SK9 1PA,385500,379500,0.195552,635196,7.980608,01
2,Q44,01C,N81002,0101021B0AAAPAP,Sod Alginate/Pot Bicarb_Tab Chble 500mg,1,3.07,2.87,60.0,201912,13040.0,SK9 1PA,385500,379500,0.004601,635196,7.980608,01
3,Q44,01C,N81002,0101021B0BEACAH,Gaviscon_Liq Orig Aniseed Relief,1,7.11,6.62,600.0,201912,13040.0,SK9 1PA,385500,379500,0.046012,635196,7.980608,01
4,Q44,01C,N81002,0101021B0BEADAJ,Gaviscon Infant_Oral Pdr Sach,3,33.74,31.40,210.0,201912,13040.0,SK9 1PA,385500,379500,0.016104,635196,7.980608,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9117201,Q70,99M,J82142,23920109202,Hollister_Urost Drain Tube Adapt,1,72.63,67.51,30.0,201912,9714.0,GU12 4DN,489500,153500,0.003088,791240,9.208259,23
9117202,Q70,99M,J82142,23941010101,Coloplast_SenSura Mio Concave Click B/Plt 50mm...,1,40.48,37.63,10.0,201912,9714.0,GU12 4DN,489500,153500,0.001029,791240,9.208259,23
9117203,Q70,99M,J82142,23960109725,Hollister_Moderma Flex Urost Bag Transpt 15-55mm,1,209.64,194.86,40.0,201912,9714.0,GU12 4DN,489500,153500,0.004118,791240,9.208259,23
9117204,Q70,99M,J82142,23960109781,Hollister_Moderma Flex Flextend Urost Bag Tran...,1,193.59,179.94,30.0,201912,9714.0,GU12 4DN,489500,153500,0.003088,791240,9.208259,23


In [11]:
from matplotlib import pyplot
asthma = prescription_data.loc[prescription_data["BNF CHAPTER"] == "03"]
asthma = asthma.loc[prescription_data["RATE"] > 1]
pyplot.scatter(asthma['pm252019g'].values, asthma['RATE'].values)

In [12]:
import scipy.stats
asthma = asthma.dropna()
scipy.stats.pearsonr(asthma['pm252019g'].values, asthma['RATE'].values)

(0.05546284284915973, 0.20237394149428892)

In [13]:
asthma['pm252019g'].values

array([ 8.162023,  8.162023,  8.162023,  8.162023,  8.162023,  8.162023,
        8.162023,  8.162023,  8.162023,  8.162023,  8.162023,  8.162023,
        8.162023,  8.955935,  8.955935,  9.355621,  8.846985,  8.323943,
        8.131237,  7.819715,  6.142898,  6.978452,  6.682895,  6.682895,
        5.652047,  6.421986,  6.421986,  6.232037,  7.154654,  7.154654,
        6.538114,  6.538114,  5.837515,  5.837515,  6.542742,  6.542742,
        5.251597,  7.643338,  7.643338,  7.643338,  7.643338,  7.643338,
        7.643338,  6.914685,  6.914685,  6.914685,  6.914685,  6.914685,
        6.914685,  7.354345,  7.354345,  7.484829,  7.322595,  7.322595,
        7.175265,  7.175265,  7.804417,  7.804417,  7.804417,  7.804417,
        8.65485 ,  9.002939,  9.002939,  9.002939,  8.300234,  9.029774,
        9.029774,  9.029774,  8.801208,  9.320744,  7.174757,  9.774611,
        9.385372,  9.385372,  8.950987,  9.282384,  9.282384,  9.282384,
       10.10625 , 10.10625 ,  9.610066, 10.08155 , 

In [14]:
practice_data

,Practice,Code,Total List Size
0,ABBEY MEDICAL CENTRE,F82625,"6,929.00"
1,AURORA MEDCARE,F82042,"9,097.00"
2,BARKING MEDICAL GROUP PRACTICE,F82018,"11,357.00"
3,DR A ARIF,F86040,"4,965.00"
4,DR A MOGHAL'S PRACTICE,F82677,"6,941.00"
...,...,...,...
6823,WEST END SURGERY,J82008,"7,782.00"
6824,WEST MEON SURGERY,J82036,"2,682.00"
6825,WHITCHURCH SURGERY,J82214,"9,118.00"
6826,WICKHAM SURGERY,J82034,"12,991.00"


In [15]:
prescription_data["BNF CODE"][6266000]

'0303020G0AAABAB'